In [ ]:
import os
import json
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Cluster setup

In [ ]:
tf_config = {
    'cluster': {'evaluator': ['192.168.1.6:12345']},
    'task': {'type': 'evaluator', 'index': 0}
}
os.environ.pop('TF_CONFIG', None)
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
if cluster_resolver.task_type != 'evaluator':
    raise SystemError('Machine is in wrong role')

# Path setup

In [ ]:
TRAIN_PATH = 'Dataset/Train'
VALIDATE_PATH = 'Dataset/Validate'
TEST_PATH = 'Dataset/Test'

In [ ]:
MODEL_PATH = 'Model'

BASE_MODEL_CKPT = os.path.join(MODEL_PATH, 'base_model_ckpt-{epoch}')
BASE_MODEL_TRAINED = os.path.join(MODEL_PATH, 'base_model_trained.hdf5')
BASE_MODEL_FIG = os.path.join(MODEL_PATH, 'base_model_fig.jpg')
BASE_MODEL_LOG = os.path.join(MODEL_PATH, 'base_model_log')
BASE_MODEL_BACKUP = os.path.join(MODEL_PATH, 'base_model_backup')

FINE_TUNE_MODEL_CKPT = os.path.join(MODEL_PATH, 'fine_tune_model_ckpt-{epoch}')
FINE_TUNE_MODEL_TRAINED = os.path.join(MODEL_PATH, 'fine_tune_model_trained.hdf5')
FINE_TUNE_MODEL_FIG = os.path.join(MODEL_PATH, 'fine_tune_model_fig.jpg')
FINE_TUNE_MODEL_LOG = os.path.join(MODEL_PATH, 'fine_tune_model_log')
FINE_TUNE_MODEL_BACKUP = os.path.join(MODEL_PATH, 'fine_tune_backup')

# Preparing data

In [ ]:
CLASSES = 30
IMAGE_SIZE = (300, 300)
BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
validate_generator = ImageDataGenerator(rescale=1./255)
generated_validate_data = validate_generator.flow_from_directory(
    VALIDATE_PATH, 
    target_size = IMAGE_SIZE, 
    batch_size = GLOBAL_BATCH_SIZE
)

In [ ]:
validate_dataset = tf.data.Dataset.from_generator(
    lambda: generated_validate_data, 
    output_types = (tf.float32, tf.float32), 
    output_shapes = (
        [GLOBAL_BATCH_SIZE, *IMAGE_SIZE, 3], 
        [GLOBAL_BATCH_SIZE, CLASSES]
    )
).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Define the model

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

In [ ]:
pretrained_model = ResNet152V2(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)

# Side-car evaluation

## Stage 1: Transfer learning

In [ ]:
for layer in pretrained_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
tf.keras.experimental.SidecarEvaluator(
    model = model,
    data = data,
    checkpoint_dir = BASE_MODEL_CKPT, # Dir for training-saved checkpoint
    steps = None, # Evaluate until dataset is exhausted
    max_evaluations = None, # The evaluation needs to be stopped manually
    callbacks = [TensorBoard(log_dir=BASE_MODEL_LOG)]
).start()

## Stage 2: Fine tuning

In [ ]:
from tensorflow.keras.optimizers import SGD
for layer in pretrained_model.layers[:FINE_TUNE_AT]: layer.trainable = False
for layer in pretrained_model.layers[FINE_TUNE_AT:]: layer.trainable = True
model.compile(
    optimizer = SGD(learning_rate=1e-4, momentum=0.9), 
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)

In [ ]:
tf.keras.experimental.SidecarEvaluator(
    model = model,
    data = data,
    checkpoint_dir = FINE_TUNE_MODEL_CKPT, # Dir for training-saved checkpoint
    steps = None, # Evaluate until dataset is exhausted
    max_evaluations = None, # The evaluation needs to be stopped manually
    callbacks = [TensorBoard(log_dir=BASE_MODEL_LOG)]
).start()